In [1]:
!pip install datasets

!pip install transformers torch
!pip install accelerate
!pip install accelerate
!pip install -U bitsandbytes
!pip install trl
!pip install peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

dataset = load_dataset("gretelai/synthetic_text_to_sql")

# Print dataset structure
print(dataset)

README.md:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 5851
    })
})


In [3]:
# First, filter the dataset by domain
filtered_dataset = {
    split: dataset[split].filter(lambda example: example["domain"] == "wildlife conservation")
    for split in dataset.keys()
}

# Select only the desired columns
columns_to_keep = ['sql_prompt', 'sql_context', 'sql', 'sql_explanation']
filtered_dataset['train'] = filtered_dataset['train'].select_columns(columns_to_keep)
filtered_dataset['test'] = filtered_dataset['test'].select_columns(columns_to_keep)

print("Filtered dataset:", filtered_dataset)

# Define the formatting function
def format_instruction(example):
    return {
        "text": f"### Instruction: {example['sql_prompt']}\n\n### Context: {example['sql_context']}\n\n### Response: {example['sql']}\n\n### Explanation: {example['sql_explanation']}"
    }

# Apply formatting to your filtered dataset
formatted_dataset = {
    split: filtered_dataset[split].map(format_instruction)
    for split in filtered_dataset.keys()
}

print("Formatted dataset:", formatted_dataset)

# Now you can proceed with tokenization
from transformers import AutoTokenizer

# Load your tokenizer

model_id = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Tokenize the formatted dataset
tokenized_dataset = {
    split: formatted_dataset[split].map(tokenize_function, batched=True)
    for split in formatted_dataset.keys()
}

print("Tokenized dataset ready for training:", tokenized_dataset)


Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5851 [00:00<?, ? examples/s]

Filtered dataset: {'train': Dataset({
    features: ['sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
    num_rows: 996
}), 'test': Dataset({
    features: ['sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
    num_rows: 52
})}


Map:   0%|          | 0/996 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Map:   0%|          | 0/996 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Tokenized dataset ready for training: {'train': Dataset({
    features: ['sql_prompt', 'sql_context', 'sql', 'sql_explanation', 'text', 'input_ids', 'attention_mask'],
    num_rows: 996
}), 'test': Dataset({
    features: ['sql_prompt', 'sql_context', 'sql', 'sql_explanation', 'text', 'input_ids', 'attention_mask'],
    num_rows: 52
})}


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
import torch

# Configure quantization for memory efficiency (optional but recommended)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.pad_token = tokenizer.eos_token

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)



config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [5]:
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig

# Create proper SFTConfig with max_seq_length
training_args = SFTConfig(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=200,
    max_seq_length=512  # Move this parameter here
)

# Configure LoRA for parameter-efficient fine-tuning
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

# Initialize the SFTTrainer with the loaded model and peft_config
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"] if "test" in tokenized_dataset else None,
    tokenizer=tokenizer,
    peft_config=peft_config # Add this parameter
    # packing=True  # Optional: for efficient sequence packing
)


<ipython-input-5-4ea600bdda8b>:28: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/996 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/996 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/996 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/52 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/52 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/52 [00:00<?, ? examples/s]

In [6]:
from google.colab import userdata
userdata.get('WANDB_API_KEY')

'd296aef3ed876ad33c24d1b2be252843a6bdcb75'

In [7]:
# 4. Start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dumbre-si (dumbre-si-northeastern-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.939200
20,0.646600
30,0.595000
40,0.537900
50,0.486800
60,0.471700
70,0.455000
80,0.421500
90,0.415500


TrainOutput(global_step=93, training_loss=0.547889103171646, metrics={'train_runtime': 957.365, 'train_samples_per_second': 3.121, 'train_steps_per_second': 0.097, 'total_flos': 8779911313489920.0, 'train_loss': 0.547889103171646})

In [10]:
trainer.save_model("./fine_tuned_llama")

In [18]:
#testing
from transformers import pipeline

# Load your fine-tuned model
fine_tuned_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_llama")
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_llama")

# Create a text generation pipeline
generator = pipeline("text-generation", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

# Test with a prompt
test_prompt = """### Instruction: Write a SQL query to find all endangered species with a population of less than 1000 individuals.

### Context: CREATE TABLE endangered_species (
    species_id INT PRIMARY KEY,
    scientific_name VARCHAR(100),
    common_name VARCHAR(100),
    conservation_status VARCHAR(50),
    habitat_type VARCHAR(100),
    population_count INT,
    year_assessed INT,
    primary_threats VARCHAR(200)
);"""
response = generator(test_prompt, max_length=500, num_return_sequences=1)
print(response[0]['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


### Instruction: Write a SQL query to find all endangered species with a population of less than 1000 individuals.

### Context: CREATE TABLE endangered_species (
    species_id INT PRIMARY KEY,
    scientific_name VARCHAR(100),
    common_name VARCHAR(100),
    conservation_status VARCHAR(50),
    habitat_type VARCHAR(100),
    population_count INT,
    year_assessed INT,
    primary_threats VARCHAR(200)
); INSERT INTO endangered_species (species_id, scientific_name, common_name, conservation_status, habitat_type, population_count, year_assessed, primary_threats) VALUES (1, 'Amur Leopard', 'Amur Leopard', 'Vulnerable', 'Savanna', 200, 2020, 'Habitat loss and poaching'), (2, 'Javan Rhino', 'Javan Rhino', 'Critically Endangered', 'Forest', 50, 2019, 'Habitat destruction and poaching'), (3, 'Giant Panda', 'Giant Panda', 'Endangered', 'Mountain', 1800, 2019, 'Habitat loss and poaching');

### Response: SELECT scientific_name, common_name, conservation_status, population_count FROM endange

In [15]:
from google.colab import files
files.download('./fine_tuned_llama/adapter_config.json')
files.download('./fine_tuned_llama/adapter_model.safetensors')
files.download('./fine_tuned_llama/tokenizer_config.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>